<a href="https://colab.research.google.com/github/IvanEFibiger/BDyML/blob/main/LABORATORIO_DE_DATOS_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Librerías importadas

In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


## DataFrames importados

In [ ]:
ruta_de_archivos = '/content/drive/MyDrive/LABORATORIO 1 ML y BD'

archivos = os.listdir(ruta_de_archivos)

dataframes= {}

for archivo in archivos:
    if archivo.endswith('.csv'):  # Asegurarse de que solo se consideren archivos CSV
        mes = archivo.split('_')[3].split('.')[0]  # Obtener el nombre del archivo sin la extensión
        ruta_completa = os.path.join(ruta_de_archivos, archivo)  # Ruta completa al archivo
        df = pd.read_csv(ruta_completa)  # Cargar el archivo CSV en un DataFrame
        df_ventas = df.copy()

        df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
        df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')


        df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
        df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

        # Elimino todos los NaN y me quedo solo con los valores numericos
        df_ventas = df_ventas.dropna()

        # Transformo todos los valores numericos al tipo de dato int
        df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
        df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(int)

        dataframes[mes] = df_ventas  # Agregar el DataFrame al diccionario usando el nombre del mes



In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes[mes]



##Extraer datos de fecha y hora

In [ ]:
def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  datos['Mes'] = datos['Fecha de Pedido'].dt.month
  datos['Año'] = datos['Fecha de Pedido'].dt.year
  return datos

dataframe_mes = extraer_datos(dataframe_mes)

##Total de ventas por mes

In [ ]:
# Creo listas para almacenar los totales de ventas y los nombres de los meses
totales_ventas = []
nombres_meses = []

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)

    total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()
    totales_ventas.append(total_ventas_mes)
    nombres_meses.append(mes)

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=nombres_meses,
    y=totales_ventas,
    marker=dict(color=totales_ventas, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=totales_ventas,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Unidades Vendidas"),
)

fig.show()



Porcentaje de ventas por mes


In [ ]:

# Asegúrate de que los meses estén en el orden correcto
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
indices_ordenados = [meses.index(mes) for mes in sorted(meses, key=meses.index)]
totales_ventas_ordenadas = [totales_ventas[i] for i in indices_ordenados]

# Calcula el porcentaje de ventas para cada mes
porcentaje_ventas = [(total / sum(totales_ventas_ordenadas)) * 100 for total in totales_ventas_ordenadas]

# Crea un gráfico de torta
fig = go.Figure(data=[go.Pie(labels=meses, values=porcentaje_ventas, sort=False)])

# Configura el diseño del gráfico
fig.update_layout(title="Porcentaje de Ventas por Mes")

fig.show()

##Total de ventas por hora, en diferentes meses

In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites
    Ventas_por_hora = dataframe_mes.groupby('Hora')['Cantidad Pedida'].count()



    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=Horas,
        y=Ventas_por_hora,
        mode='lines',
        name=mes,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora en diferentes meses",
    xaxis=dict(title="Hora del día", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()



##Total de ventas por hora

In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()
horas_analizadas = [19, 12]

for hora in horas_analizadas:
    total_productos_vendidos = []

    for mes in meses:
        dataframe_mes = dataframes[mes]
        dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites

        ventas_hora = dataframe_mes[dataframe_mes['Hora'] == hora]

        # Calcular la cantidad total de productos vendidos para la hora actual
        total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].count())

    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=meses,
        y=total_productos_vendidos,
        mode='lines',
        name=f'{hora} horas',
        line=dict(width=2, shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora",
    xaxis=dict(title="Meses", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

##Ventas por estado

In [ ]:
# Combinar todos los DataFrames en uno solo
df_combined = pd.concat(dataframes.values(), ignore_index=True)

# Dividir la columna 'Dirección de Envio' y agregar columnas al DataFrame
direccion_df = df_combined['Dirección de Envio'].str.split(', ', expand=True)
direccion_df.columns = ['Calle', 'Ciudad', 'PO BOX']
direccion_df[['Estado', 'Cod Postal']] = direccion_df['PO BOX'].str.split(' ', expand=True)
df_combined = pd.concat([df_combined, direccion_df], axis=1)

# Calcular las ventas por estado
ventas_por_estado = df_combined.groupby('Estado')['Cantidad Pedida'].sum().reset_index()

# Crear el choropleth map
fig = px.choropleth(ventas_por_estado, locations='Estado', locationmode='USA-states', color='Cantidad Pedida',
                    color_continuous_scale='Plotly3', scope='usa',
                    title='Ventas por estado')

# Agregar las etiquetas de los estados
fig.add_trace(go.Scattergeo(
    locations=ventas_por_estado['Estado'],
    locationmode='USA-states',
    text=ventas_por_estado['Estado'],
    mode='text',
    textfont=dict(color='white')
))

fig.show()

In [ ]:
# Filtra y selecciona las columnas relevantes
df_ventas_estado_mes = df_combined[['Cantidad Pedida', 'Mes', 'Estado']]

# Agrupa los datos por Mes y Estado y calcula los totales de ventas
ventas_por_estado_mes = df_ventas_estado_mes.groupby(['Mes', 'Estado'])['Cantidad Pedida'].sum().reset_index()

# Crea el gráfico de líneas para representar la variación en las ventas por estado a lo largo de los meses
fig = px.line(ventas_por_estado_mes, x='Mes', y='Cantidad Pedida', color='Estado',
              title='Variación en las Ventas por Estado a lo largo de los Meses',
              labels={'Cantidad Pedida': 'Total de Ventas'},
              line_shape='spline')
fig.show()

In [ ]:
df_combined

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora,Dia,Mes,Año,Calle,Ciudad,PO BOX,Estado,Cod Postal
0,176558,USB-C Charging Cable,2,11,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",8,19,4,2019,917 1st St,Dallas,TX 75001,TX,75001
1,176559,Bose SoundSport Headphones,1,99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",22,7,4,2019,682 Chestnut St,Boston,MA 02215,MA,02215
2,176560,Google Phone,1,600,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",14,12,4,2019,669 Spruce St,Los Angeles,CA 90001,CA,90001
3,176560,Wired Headphones,1,11,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",14,12,4,2019,669 Spruce St,Los Angeles,CA 90001,CA,90001
4,176561,Wired Headphones,1,11,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",9,30,4,2019,333 8th St,Los Angeles,CA 90001,CA,90001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2,2019-09-17 20:56:00,"840 Highland St, Los Angeles, CA 90001",20,17,9,2019,840 Highland St,Los Angeles,CA 90001,CA,90001
185946,259354,iPhone,1,700,2019-09-01 16:00:00,"216 Dogwood St, San Francisco, CA 94016",16,1,9,2019,216 Dogwood St,San Francisco,CA 94016,CA,94016
185947,259355,iPhone,1,700,2019-09-23 07:39:00,"220 12th St, San Francisco, CA 94016",7,23,9,2019,220 12th St,San Francisco,CA 94016,CA,94016
185948,259356,34in Ultrawide Monitor,1,379,2019-09-19 17:30:00,"511 Forest St, San Francisco, CA 94016",17,19,9,2019,511 Forest St,San Francisco,CA 94016,CA,94016


4. Análisis del producto más vendido:

In [ ]:
df_ventas_relevantes = df_combined[['Producto', 'Cantidad Pedida']]
df_ventas_relevantes = df_ventas_relevantes.groupby(['Producto'])['Cantidad Pedida'].sum().reset_index()
df_ventas_relevantes = df_ventas_relevantes.sort_values('Cantidad Pedida', ascending=False)
df_ventas_relevantes

,Producto,Cantidad Pedida
5,AAA Batteries (4-pack),31017
4,AA Batteries (4-pack),27635
15,USB-C Charging Cable,23975
12,Lightning Charging Cable,23217
17,Wired Headphones,20557
6,Apple Airpods Headphones,15661
7,Bose SoundSport Headphones,13457
2,27in FHD Monitor,7550
18,iPhone,6849
1,27in 4K Gaming Monitor,6244


In [ ]:
# Calcular el total de ventas
total_ventas = df_ventas_relevantes['Cantidad Pedida'].sum()

# Calcular el porcentaje de ventas para cada producto y redondearlo a 2 decimales
df_ventas_relevantes['Porcentaje Ventas'] = (df_ventas_relevantes['Cantidad Pedida'] / total_ventas * 100).round(2)

fig = go.Figure(data=[go.Table(
    header=dict(values=['Producto', 'Cantidad Pedida', 'Porcentaje Ventas']),
    cells=dict(values=[df_ventas_relevantes['Producto'], df_ventas_relevantes['Cantidad Pedida'], df_ventas_relevantes['Porcentaje Ventas']])
)])

# Personalizar el diseño de la tabla (opcional)
fig.update_layout(
    title='Tabla de Ventas',
    font=dict(size=12),
    margin=dict(l=5, r=5, t=30, b=10)
)

# Mostrar la tabla
fig.show()

In [ ]:
productos_mas_vendidos_por_mes = []

# Iterar a través de los meses
for mes in meses:
    # Filtrar el DataFrame combinado para el mes actual
    df_mes_actual = df_combined[df_combined['Mes'] == meses.index(mes) + 1]

    # Agrupar por producto y sumar la cantidad pedida
    ventas_por_producto = df_mes_actual.groupby('Producto')['Cantidad Pedida'].sum().reset_index()

    # Encontrar el producto más vendido en el mes actual
    producto_mas_vendido = ventas_por_producto.loc[ventas_por_producto['Cantidad Pedida'].idxmax()]

    # Agregar el producto más vendido a la lista
    productos_mas_vendidos_por_mes.append([mes, producto_mas_vendido['Producto'], producto_mas_vendido['Cantidad Pedida']])

# Crear un nuevo DataFrame con los datos
df_productos_mas_vendidos_por_mes = pd.DataFrame(productos_mas_vendidos_por_mes, columns=['Mes', 'Producto Más Vendido', 'Cantidad Vendida'])


df_productos_mas_vendidos_por_mes

,Mes,Producto Más Vendido,Cantidad Vendida
0,Enero,AAA Batteries (4-pack),1600
1,Febrero,AAA Batteries (4-pack),1972
2,Marzo,AAA Batteries (4-pack),2479
3,Abril,AAA Batteries (4-pack),2939
4,Mayo,AAA Batteries (4-pack),2927
5,Junio,AAA Batteries (4-pack),2162
6,Julio,AAA Batteries (4-pack),2365
7,Agosto,AAA Batteries (4-pack),2021
8,Septiembre,AAA Batteries (4-pack),1923
9,Octubre,AAA Batteries (4-pack),3363


In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x= df_productos_mas_vendidos_por_mes['Mes'],
    y= df_productos_mas_vendidos_por_mes['Cantidad Vendida'],
    marker=dict(color=totales_ventas, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas por mes", x=1.0, thickness=15)),  # Color y barra de colores
    text= df_productos_mas_vendidos_por_mes['Cantidad Vendida'],
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas de Baterías AAA por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()


In [ ]:
#extraigo valore del dataframe para realizar el grafico
Analisis_de_productos = df_combined.groupby('Producto').agg({
    'Cantidad Pedida':'sum',
    'Precio Unitario': 'mean'
})

#aplicamos esto para que producto no quede como indice. oredeno de mayor a menor
Analisis_de_productos = Analisis_de_productos.sort_values(by = 'Cantidad Pedida', ascending = False)
#reset de indice
Analisis_de_productos = Analisis_de_productos.reset_index(inplace=False)

Analisis_de_productos

,Producto,Cantidad Pedida,Precio Unitario
0,AAA Batteries (4-pack),31017,2.0
1,AA Batteries (4-pack),27635,3.0
2,USB-C Charging Cable,23975,11.0
3,Lightning Charging Cable,23217,14.0
4,Wired Headphones,20557,11.0
5,Apple Airpods Headphones,15661,150.0
6,Bose SoundSport Headphones,13457,99.0
7,27in FHD Monitor,7550,149.0
8,iPhone,6849,700.0
9,27in 4K Gaming Monitor,6244,389.0


In [ ]:
Productos = Analisis_de_productos['Producto']
Unidades = Analisis_de_productos['Cantidad Pedida']
Precio = Analisis_de_productos['Precio Unitario']


# Crear el objeto figura
fig = go.Figure()

fig.add_trace(go.Bar(
        x = Productos,
        y= Unidades,
        name = 'Unidades Vendidas',
        marker = dict(color= 'Light Blue'),
        yaxis= 'y'
)
)

fig.add_trace(go.Scatter(
        x = Productos,
        y= Precio,
        name = 'Precio unitario',
        line = dict(color= 'Red', shape='spline', smoothing= 1.3) ,
        yaxis= 'y2'
        )
)


fig.update_layout(
    title = 'Unidades Vendidas y Precios de Productos',
    xaxis = dict(title='Productos'),
    yaxis = dict(title='Unidades', side='left'),
    yaxis2 = dict(title = 'Precio', overlaying = 'y', side='right'),
    yaxis_range = [0, max(Unidades)*1.1],
    yaxis2_range = [0, max(Precio)*1.1]

)

In [ ]:
df_combined

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora,Dia,Mes,Año,Calle,Ciudad,PO BOX,Estado,Cod Postal
0,176558,USB-C Charging Cable,2,11,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",8,19,4,2019,917 1st St,Dallas,TX 75001,TX,75001
1,176559,Bose SoundSport Headphones,1,99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",22,7,4,2019,682 Chestnut St,Boston,MA 02215,MA,02215
2,176560,Google Phone,1,600,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",14,12,4,2019,669 Spruce St,Los Angeles,CA 90001,CA,90001
3,176560,Wired Headphones,1,11,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",14,12,4,2019,669 Spruce St,Los Angeles,CA 90001,CA,90001
4,176561,Wired Headphones,1,11,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",9,30,4,2019,333 8th St,Los Angeles,CA 90001,CA,90001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2,2019-09-17 20:56:00,"840 Highland St, Los Angeles, CA 90001",20,17,9,2019,840 Highland St,Los Angeles,CA 90001,CA,90001
185946,259354,iPhone,1,700,2019-09-01 16:00:00,"216 Dogwood St, San Francisco, CA 94016",16,1,9,2019,216 Dogwood St,San Francisco,CA 94016,CA,94016
185947,259355,iPhone,1,700,2019-09-23 07:39:00,"220 12th St, San Francisco, CA 94016",7,23,9,2019,220 12th St,San Francisco,CA 94016,CA,94016
185948,259356,34in Ultrawide Monitor,1,379,2019-09-19 17:30:00,"511 Forest St, San Francisco, CA 94016",17,19,9,2019,511 Forest St,San Francisco,CA 94016,CA,94016


In [ ]:
df_combined['Dia de la Semana'] = df_combined['Fecha de Pedido'].dt.dayofweek
nombres_dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
df_combined['Dia de la Semana'] = df_combined['Dia de la Semana'].map(lambda x: nombres_dias_semana[x])



In [ ]:
# Crear una lista vacía para almacenar los gráficos
figs = []

# Obtener los años únicos en el DataFrame
años = df_combined['Año'].unique()

# Iterar a través de los años
for año in años:
    # Filtrar el DataFrame combinado para el año actual
    df_año_actual = df_combined[df_combined['Año'] == año]

    # Iterar a través de los meses
    for mes in meses:
        # Filtrar el DataFrame del año actual para el mes actual
        df_mes_actual = df_año_actual[df_año_actual['Mes'] == meses.index(mes) + 1]

        # Agrupar por fecha de pedido y sumar la cantidad pedida
        ventas_por_dia = df_mes_actual.groupby(df_mes_actual['Fecha de Pedido'].dt.normalize())['Cantidad Pedida'].sum()

        # Formatear las fechas para que se muestren como 'Día (Día de la semana)'
        dias = ventas_por_dia.index.strftime('%d (%A)')

        # Crear un gráfico de líneas para el mes y año actual si ventas_por_dia no está vacío
        if not ventas_por_dia.empty:
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=dias,
                y=ventas_por_dia,
                name='Ventas por día',
                line=dict(color='Red', shape='spline', smoothing=1.3)
            ))

            # Configurar el diseño del gráfico
            fig.update_layout(
                title=f'Unidades vendidas por día en {mes} {año}',
                xaxis=dict(title='Días', tickvals=dias),
                yaxis=dict(title='Unidades Vendidas', side='left'),
                yaxis_range=[0, max(ventas_por_dia)*1.1]
            )

            # Agregar el gráfico a la lista
            figs.append(fig)

# Mostrar los gráficos
for fig in figs:
    fig.show()

In [ ]:
num_filas = len(df_combined)

print(f'Número de filas en df_combined: {num_filas}')

Número de filas en df_combined: 185950


In [ ]:
Analisis_b = Analisis_de_productos['Cantidad Pedida'].sum()
Analisis_b

209079

In [ ]:
df_combined


,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora,Dia,Mes,Año,Calle,Ciudad,PO BOX,Estado,Cod Postal,Dia de la Semana
0,176558,USB-C Charging Cable,2,11,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",8,19,4,2019,917 1st St,Dallas,TX 75001,TX,75001,Viernes
1,176559,Bose SoundSport Headphones,1,99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",22,7,4,2019,682 Chestnut St,Boston,MA 02215,MA,02215,Domingo
2,176560,Google Phone,1,600,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",14,12,4,2019,669 Spruce St,Los Angeles,CA 90001,CA,90001,Viernes
3,176560,Wired Headphones,1,11,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",14,12,4,2019,669 Spruce St,Los Angeles,CA 90001,CA,90001,Viernes
4,176561,Wired Headphones,1,11,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",9,30,4,2019,333 8th St,Los Angeles,CA 90001,CA,90001,Martes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2,2019-09-17 20:56:00,"840 Highland St, Los Angeles, CA 90001",20,17,9,2019,840 Highland St,Los Angeles,CA 90001,CA,90001,Martes
185946,259354,iPhone,1,700,2019-09-01 16:00:00,"216 Dogwood St, San Francisco, CA 94016",16,1,9,2019,216 Dogwood St,San Francisco,CA 94016,CA,94016,Domingo
185947,259355,iPhone,1,700,2019-09-23 07:39:00,"220 12th St, San Francisco, CA 94016",7,23,9,2019,220 12th St,San Francisco,CA 94016,CA,94016,Lunes
185948,259356,34in Ultrawide Monitor,1,379,2019-09-19 17:30:00,"511 Forest St, San Francisco, CA 94016",17,19,9,2019,511 Forest St,San Francisco,CA 94016,CA,94016,Jueves


In [ ]:
# Convertir 'Dia de la Semana' a 'Tipo de Dia'
df_combined['Tipo de Dia'] = df_combined['Dia de la Semana'].apply(lambda x: 'Laboral' if x in ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes'] else 'No Laboral')
# Crear una nueva columna 'Semana del Mes'
df_combined['Semana del Mes'] = df_combined['Fecha de Pedido'].apply(lambda x: (x.day-1) // 7 + 1)

# Agrupar por 'Mes', 'Semana del Mes' y 'Tipo de Dia' y contar las ventas
ventas_por_semana = df_combined.groupby(['Mes', 'Semana del Mes', 'Tipo de Dia'])['Cantidad Pedida'].sum().reset_index()

# Calcular el número de días laborales y no laborales en cada semana
dias_por_semana = df_combined.groupby(['Mes', 'Semana del Mes', 'Tipo de Dia']).size().reset_index(name='Dias')

# Fusionar 'ventas_por_semana' y 'dias_por_semana' en un solo DataFrame
df_final = pd.merge(ventas_por_semana, dias_por_semana, on=['Mes', 'Semana del Mes', 'Tipo de Dia'])

# Calcular el promedio de ventas por día
df_final['Promedio Ventas'] = df_final['Cantidad Pedida'] / df_final['Dias']

# Crear subgráficos en una cuadrícula 3x4
fig = make_subplots(rows=4, cols=3, subplot_titles=[str(i) for i in range(1, 13)])

# Definir colores para los tipos de días
colors = {'Laboral': 'orange', 'No Laboral': 'turquoise'}

# Agregar gráficos de barras a cada subgráfico
for i, mes in enumerate(range(1, 13), start=1):
    for tipo in ['Laboral', 'No Laboral']:
        fig.add_trace(
            go.Bar(x=df_final[(df_final['Mes'] == mes) & (df_final['Tipo de Dia'] == tipo)]['Semana del Mes'],
                   y=df_final[(df_final['Mes'] == mes) & (df_final['Tipo de Dia'] == tipo)]['Promedio Ventas'],
                   name=tipo,
                   marker_color=colors[tipo],  # Usar el color definido para cada tipo de día
                   showlegend = True if i == 1 else False),  # Mostrar la leyenda solo en el primer subgráfico
            row=(i-1)//3+1, col=(i-1)%3+1
        )
    # Ajustar el rango del eje y para este subgráfico basado en los datos que está mostrando
    fig.update_yaxes(title_text='Promedio de Ventas', range=[0, df_final[df_final['Mes'] == mes]['Promedio Ventas'].max()], row=(i-1)//3+1, col=(i-1)%3+1)
    fig.update_xaxes(title_text='Semana', row=(i-1)//3+1, col=(i-1)%3+1)

fig.update_layout(height=1200, width=1200,
                  title_text="Promedio de Ventas por Semana y Tipo de Día, Discriminadas por Mes"
                  )
fig.show()

In [ ]:
df_combined


,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora,Dia,Mes,Año,Calle,Ciudad,PO BOX,Estado,Cod Postal,Dia de la Semana,Tipo de Dia,Semana del Mes
0,176558,USB-C Charging Cable,2,11,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",8,19,4,2019,917 1st St,Dallas,TX 75001,TX,75001,Viernes,Laboral,3
1,176559,Bose SoundSport Headphones,1,99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",22,7,4,2019,682 Chestnut St,Boston,MA 02215,MA,02215,Domingo,No Laboral,1
2,176560,Google Phone,1,600,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",14,12,4,2019,669 Spruce St,Los Angeles,CA 90001,CA,90001,Viernes,Laboral,2
3,176560,Wired Headphones,1,11,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",14,12,4,2019,669 Spruce St,Los Angeles,CA 90001,CA,90001,Viernes,Laboral,2
4,176561,Wired Headphones,1,11,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",9,30,4,2019,333 8th St,Los Angeles,CA 90001,CA,90001,Martes,Laboral,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2,2019-09-17 20:56:00,"840 Highland St, Los Angeles, CA 90001",20,17,9,2019,840 Highland St,Los Angeles,CA 90001,CA,90001,Martes,Laboral,3
185946,259354,iPhone,1,700,2019-09-01 16:00:00,"216 Dogwood St, San Francisco, CA 94016",16,1,9,2019,216 Dogwood St,San Francisco,CA 94016,CA,94016,Domingo,No Laboral,1
185947,259355,iPhone,1,700,2019-09-23 07:39:00,"220 12th St, San Francisco, CA 94016",7,23,9,2019,220 12th St,San Francisco,CA 94016,CA,94016,Lunes,Laboral,4
185948,259356,34in Ultrawide Monitor,1,379,2019-09-19 17:30:00,"511 Forest St, San Francisco, CA 94016",17,19,9,2019,511 Forest St,San Francisco,CA 94016,CA,94016,Jueves,Laboral,3


In [ ]:
import plotly.graph_objects as go

# Calcular el ingreso para cada pedido
df_combined['Ingreso'] = df_combined['Cantidad Pedida'] * df_combined['Precio Unitario']

# Agrupar por 'Mes' y sumar los ingresos
ingresos_por_mes = df_combined.groupby('Mes')['Ingreso'].sum().reset_index()

# Crear un gráfico de barras de los ingresos por mes con Plotly
fig = go.Figure(data=[go.Bar(
    x=ingresos_por_mes['Mes'],
    y=ingresos_por_mes['Ingreso'],
    text=ingresos_por_mes['Ingreso'],
    textposition='auto',
    marker_color='coral'
)])

fig.update_layout(title_text='Ingreso Total Generado por Mes',
                  xaxis=dict(title='Mes'),
                  yaxis=dict(title='Ingresos en u$s', side='left'))
fig.show()

In [ ]:
# Calcular la cantidad total de ventas para cada ciudad
ventas_por_ciudad = df_combined.groupby('Ciudad')['Cantidad Pedida'].sum().reset_index()

# Ordenar las ciudades por ventas
ventas_por_ciudad = ventas_por_ciudad.sort_values('Cantidad Pedida', ascending=False)

# Calcular el total de ventas
total_ventas = df_combined['Cantidad Pedida'].sum()

# Calcular el porcentaje de ventas para cada ciudad
ventas_por_ciudad['Porcentaje Ventas'] = (ventas_por_ciudad['Cantidad Pedida'] / total_ventas) * 100

# Crear un gráfico de barras con Plotly, mostrando la cantidad total de ventas por ciudad
fig = go.Figure(data=[go.Bar(
    x=ventas_por_ciudad['Ciudad'],
    y=ventas_por_ciudad['Cantidad Pedida'],
    text=ventas_por_ciudad.apply(lambda row: f"Ventas: {row['Cantidad Pedida']}<br>Porcentaje: {row['Porcentaje Ventas']:.2f}%", axis=1),
    textposition='inside',
    insidetextanchor='start',
    marker_color=px.colors.qualitative.Plotly  # Usar una paleta de colores diferente para cada barra
)])

fig.update_layout(title_text='Cantidad Total de Ventas por Ciudad',
                  xaxis=dict(title='Ciudad'),
                  yaxis=dict(title='Unidades Vendidas', side='left'))
fig.show()